In [67]:
from scipy import stats
import seaborn as sns
import pandas as pd
from scipy.stats import norm
import numpy as np
import statistics

# Exercício 1

In [68]:
pop = stats.expon.rvs(scale=300, size = 10000)
mediana = np.median(pop)

In [70]:
n = 30
t = stats.t(df=n).ppf((0.05, 0.95))
estimativas = []
p = 0
mds = []
for i in range(100):
    amostra = pop[stats.randint.rvs(0,len(pop),size=n)]
    md = np.median(amostra)
    var = amostra.var()
    ic = [md + t*(var/n-1)**(1/2)]
    estimativas.append(ic)
    mds.append(md)
    if mediana >= ic[0][0] and mediana <= ic[0][1]:
        p = p + 1
    ic_mediana_bootstrap = np.median(estimativas) + t*(var/n-1)**(1/2)

print("Proporção de intervalos que contiveram o parâmetro: ", p/100, "\n Intervalo de confiança 90% bootstrap para a mediana da população: ", ic_mediana_bootstrap)



Proporção de intervalos que contiveram o parâmetro:  0.93 
 Intervalo de confiança 90% bootstrap para a mediana da população:  [107.9719639  318.55058678]


## Em ambos os casos, o verdadeiro valor foi bem estimado pelo método bootstrap.
### No caso do intervalo pontual, porém, apesar da estimativa pontual estar bem próxima ao verdadeiro valor, a amplitude do intervalo ainda é um pouco grande demais..

# Exercício 2

In [72]:
data = pd.read_csv('https://gist.githubusercontent.com/seankross/a412dfbd88b3db70b74b/raw/5f23f993cd87c283ce766e7ac6b329ee7cc2e1d1/mtcars.csv')
data.head(3)

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1


In [126]:
valores = data.groupby("am")["mpg"].mean()
dif = data.groupby("am")["mpg"].mean()[1] - data.groupby("am")["mpg"].mean()[0]
print("Médias:", valores, "\n Diferença:", dif)


Médias: am
0    17.147368
1    24.392308
Name: mpg, dtype: float64 
 Diferença: 7.24493927125506


In [130]:
difs = []
for i in range(1000):
    amostra = data["mpg"].sample(n=19, replace=False)
    difs.append(amostra.mean() - data["mpg"].drop(amostra.index).mean())
med = np.array(difs).mean()

print("Média das diferenças:", med,"\n Diferenças:" , difs)

Média das diferenças: -0.06071902834008206 
 Diferenças: [-2.308906882591092, 0.15263157894736779, 2.2384615384615394, 0.17854251012145994, -3.0085020242915057, -4.291093117408902, 2.355060728744938, 1.202024291497974, -1.2595141700404895, 0.5024291497975675, 0.3210526315789437, -0.3785425101214557, -0.46923076923076934, -3.2676113360323917, 1.1761133603238818, -3.3453441295546575, -0.43036437246963644, -0.7672064777327989, -0.6117408906882638, 1.5, 0.12672064777327918, 1.888663967611329, -1.6740890688259071, 2.329149797570853, -2.6716599190283397, 2.873279352226721, 2.1348178137651814, 1.5259109311740886, -0.1971659919028319, -1.0522267206477736, -3.0344129554655837, -3.099190283400816, 0.29514170040485865, -1.6093117408906927, -0.8060728744939247, -1.9979757085020289, -4.019028340080968, -0.2360323886639648, -3.2157894736842074, 3.521052631578943, 0.5672064777327925, -1.8684210526315823, -3.189878542510119, 0.38582995951416876, 2.018218623481779, 2.1348178137651814, -3.50080971659918

Daqui, vemos que existe diferença entre dividir aleatoriamente um grupo com n=13 e outro com n=19 e calcular a média do mpg, de selecionar a amostra em que am = 0 e am = 1 e calcular essas médias para cada amostra. Ou seja, o valor de am aparenta influenciar diretamente na média de mpg, rejeitando assim empiricamente a hipótese nula proposta, de igualdade de médias entre carros automáticos e manuais.